# Unitary ansatz entering the VQE

The final energy output of a VQE calculation will crucially depend on the ansatz/form of the parameterized unitary $\hat U(\boldsymbol{\theta})$ employed in state preparation. Here we review two popular approaches, the unitary coupled cluster and qubit coupled cluster methodologies, and benchmark them for energy calculations of small molecules.

In [1]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients

## Unitary Coupled Cluster (UCC)

The UCC ansatz is obtained by 'unitarizing' the traditional coupled cluster ansatz,
$$ e^{\hat T} \rightarrow e^{\hat T - \hat T^\dagger} \equiv \hat U_{\text{UCC}}$$


Due to non-commutativity of terms in $\hat T - \hat T^\dagger$, the UCC ansatz does not have a straightforward decomposition in terms of circuit primitives implementable on the quantum computer. Therefore, to obtain a form which can be compiled, we employ the Trotter approximation. The accuracy of the circuit ansatz relative to the exact UCC operator will be dependent on how many Trotter steps are employed. The number of Trotter steps is commonly set to its minimal value of one to avoid excessive circuit depth.

In [2]:
trotter_steps = 1

### LiH in STO-3G basis

Below is a sample VQE simulation using the UCCSD ansatz compiled using a single trotter step for LiH in minimal basis at $R=2.5$ (Angstrom). For comparison, we can run FCI to obtain the true ground state energy.


In [3]:
xyz_data = get_molecular_data('lih', geometry=2.5, xyz_format=True)
basis='sto-3g'

lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*lih.n_orbitals))

E_FCI = lih.compute_energy(method='fci', backend='qulacs_gpu')

print('FCI energy: {}'.format(E_FCI))

Number of spin-orbitals (qubits): 12 

FCI energy: -7.823723883389346


The UCCSD VQE optimization is of the form
$$E = \min_{\boldsymbol{\theta}} \langle \text{HF} | \hat U_{\text{UCC}}^\dagger(\boldsymbol{\theta}) \hat H  \hat U_{\text{UCC}} (\boldsymbol{\theta}) | \text{HF} \rangle $$
To expedite the optimization process, we can set the initial guess of the amplitudes to zero, i.e. the optimization will begin at the Hartree Fock state. This heuristic is best suited for when Hartree Fock is believed to be in qualitative agreement with the true ground state. To further alleviate quantum resources, we can estimate the amplitudes using classical electronic structure methods (here, MP2 perturbation theory), and only include the unitaries with non-zero estimated amplitudes.

In [8]:
H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = lih.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.optimizers.optimizer_gpyopt.minimize(objective=E, maxiter=500, initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-6, backend='qulacs_gpu') 
print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 631 terms


Number of UCCSD amplitudes: 24 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_gpyopt.OptimizerGPyOpt'> 
backend         : qulacs_gpu
samples         : None
save_history    : True
noise           : None

method          : lbfgs
Objective       : 1 expectationvalues
num acquisition: 1, time elapsed: 0.74s
num acquisition: 2, time elapsed: 1.51s
num acquisition: 3, time elapsed: 2.26s
num acquisition: 4, time elapsed: 2.98s
num acquisition: 5, time elapsed: 3.75s
num acquisition: 6, time elapsed: 4.51s
num acquisition: 7, time elapsed: 5.24s
num acquisition: 8, time elapsed: 5.97s
num acquisition: 9, time elapsed: 6.69s
num acquisition: 10, time elapsed: 7.44s
num acquisition: 11, time elapsed: 8.19s
num acquisition: 12, time elapsed: 8.94s
num acquisition: 13, time elapsed: 9.68s
num acquisition: 14, time elapsed: 10.42s
num acquisition: 15, time elapsed: 11.13s
num acquisition: 16, time elapsed: 11.84s
num acquisition: 17, time el

num acquisition: 185, time elapsed: 334.32s
num acquisition: 186, time elapsed: 336.52s
num acquisition: 187, time elapsed: 339.25s
num acquisition: 188, time elapsed: 342.48s
num acquisition: 189, time elapsed: 344.78s
num acquisition: 190, time elapsed: 347.00s
num acquisition: 191, time elapsed: 350.33s
num acquisition: 192, time elapsed: 352.48s
num acquisition: 193, time elapsed: 354.98s
num acquisition: 194, time elapsed: 358.27s
num acquisition: 195, time elapsed: 361.24s
num acquisition: 196, time elapsed: 364.13s
num acquisition: 197, time elapsed: 365.84s
num acquisition: 198, time elapsed: 367.73s
num acquisition: 199, time elapsed: 369.55s
num acquisition: 200, time elapsed: 371.59s
num acquisition: 201, time elapsed: 373.30s
num acquisition: 202, time elapsed: 375.30s
num acquisition: 203, time elapsed: 377.34s
num acquisition: 204, time elapsed: 379.65s
num acquisition: 205, time elapsed: 381.84s
num acquisition: 206, time elapsed: 384.00s
num acquisition: 207, time elaps

num acquisition: 372, time elapsed: 886.50s
num acquisition: 373, time elapsed: 890.46s
num acquisition: 374, time elapsed: 897.55s
num acquisition: 375, time elapsed: 904.45s
num acquisition: 376, time elapsed: 909.13s
num acquisition: 377, time elapsed: 913.43s
num acquisition: 378, time elapsed: 918.11s
num acquisition: 379, time elapsed: 922.86s
num acquisition: 380, time elapsed: 927.41s
num acquisition: 381, time elapsed: 931.80s
num acquisition: 382, time elapsed: 938.85s
num acquisition: 383, time elapsed: 943.66s
num acquisition: 384, time elapsed: 948.81s
num acquisition: 385, time elapsed: 953.28s
num acquisition: 386, time elapsed: 957.73s
num acquisition: 387, time elapsed: 962.44s
num acquisition: 388, time elapsed: 966.42s
num acquisition: 389, time elapsed: 970.84s
num acquisition: 390, time elapsed: 975.92s
num acquisition: 391, time elapsed: 981.33s
num acquisition: 392, time elapsed: 986.36s
num acquisition: 393, time elapsed: 990.13s
num acquisition: 394, time elaps

We see that the converged UCCSD energy is in exact agreement with the FCI energy, as expected for a $2$-electron system. 

### H4 in 6-31G basis

Now let us try a larger problem of H$_4$ in 6-31G basis. However, we will restrict the active space. The unrestricted problem leads to a $14$-qubit Hamiltonian, and $34$ UCCSD amplitudes to optimize even after neglecting the zero MP2 amplitudes. Therefore, we will remove some orbital degrees of freedom which are less important in accurately describing the electronic structure. By freezing all orbitals other than $0b_1$, $1b_1$, $2a_1$, and $3a_1$, we reduce the problem to an $8$-qubit Hamiltonian with $8$ UCCSD variational amplitudes.

In [41]:
xyz_data = get_molecular_data('h4', geometry=90, xyz_format=True)

basis = '6-31g'
active = { 'A0':[2,3]}
h4 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis)#, active_orbitals=active)
print(list(map(lambda item: item[1][0].energy, h4.orbitals_by_irrep.items())))

print('Number of spin-orbitals (qubits): {} \n'.format(2*h4.n_orbitals))

E_FCI = h4.compute_energy(method='fci', backend='qulacs_gpu')

print('FCI energy: {}'.format(E_FCI))

[-0.6577756529606307, -0.3241801225582043, -0.021997174708910133, 0.24893496000350707]
Number of spin-orbitals (qubits): 16 

FCI energy: -2.068288525356859


We will then run the UCCSD VQE simulation (***warning: tq.minimize will take several minutes - 1 hour + to converge for a VQE instance of this size.*** Smaller active spaces can be employed to lower VQE simulation runtimes).

In [42]:
H = h4.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h4.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E,  initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4, backend='qulacs_gpu', method='bfgs')

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 1473 terms


Number of UCCSD amplitudes: 40 


Starting optimization:



TypeError: minimize() missing 1 required positional argument: 'method'

In [22]:
print(result)

GPyOptResults(energy=-0.927276119007125, history=OptimizerHistory(energies=array([ 2.57738401e-01, -5.58349949e-01,  4.16314332e-01, -7.20595122e-02,
        4.97766830e-02,  3.63986893e-01,  4.41470282e-01, -4.27460046e-01,
       -3.34552874e-01, -6.88757751e-01,  3.17612575e-02,  6.29450341e-01,
        5.73832760e-01,  3.54992200e-01, -5.21438092e-01,  2.77093020e-01,
        2.55119553e-02, -2.96691072e-01, -5.85835661e-01,  4.25659865e-01,
        2.43356886e-01,  4.87073315e-01,  1.83683410e-01, -2.86054791e-01,
       -1.07986374e-01,  4.28583672e-01, -1.77113055e-01,  2.07521067e-01,
       -4.13718844e-02, -3.19489259e-01,  3.47232406e-01,  1.92401895e-01,
       -3.89797152e-01,  1.52644003e-01, -3.07677173e-01,  4.36022308e-01,
        2.24658607e-01, -2.70690690e-02, -7.84635290e-01, -2.33506975e-01,
        2.10487341e-01,  2.76904295e-01, -6.02145590e-01,  2.39555745e-01,
        3.90147986e-01,  3.21791107e-01, -5.16908220e-01, -5.44215014e-01,
       -3.99239246e-01,  

In [21]:
print('\nStarting optimization:\n')

result = tq.optimizers.optimizer_phoenics.minimize(objective=E, initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4, backend='qulacs_gpu')

print('\nObtained UCCSD energy: {}'.format(result.energy))


Starting optimization:

Phoenics config:

{'general': {'auto_desc_gen': 'False', 'batches': 5, 'boosted': 'False', 'parallel': 'False'}, 'parameters': [{'name': (3, 1, 2, 0), 'periodic': 'True', 'type': 'continuous', 'size': 1, 'low': 0, 'high': 6.283185307179586}, {'name': (5, 0, 7, 1), 'periodic': 'True', 'type': 'continuous', 'size': 1, 'low': 0, 'high': 6.283185307179586}, {'name': (3, 0, 2, 1), 'periodic': 'True', 'type': 'continuous', 'size': 1, 'low': 0, 'high': 6.283185307179586}, {'name': (5, 1, 2, 1), 'periodic': 'True', 'type': 'continuous', 'size': 1, 'low': 0, 'high': 6.283185307179586}, {'name': (3, 1, 7, 1), 'periodic': 'True', 'type': 'continuous', 'size': 1, 'low': 0, 'high': 6.283185307179586}, {'name': (5, 0, 2, 0), 'periodic': 'True', 'type': 'continuous', 'size': 1, 'low': 0, 'high': 6.283185307179586}, {'name': (2, 1, 7, 0), 'periodic': 'True', 'type': 'continuous', 'size': 1, 'low': 0, 'high': 6.283185307179586}, {'name': (4, 0, 6, 1), 'periodic': 'True', 'type'

energy = +0.24322043 , angles= {(3, 1, 2, 0): 5.304489612579346, (5, 0, 7, 1): 3.034426689147949, (3, 0, 2, 1): 3.76320219039917, (5, 1, 2, 1): 0.8069871664047241, (3, 1, 7, 1): 5.453207969665527, (5, 0, 2, 0): 2.660248279571533, (2, 1, 7, 0): 1.1727230548858643, (4, 0, 6, 1): 0.18380218744277954, (7, 0, 5, 1): 0.007298745680600405, (4, 1, 6, 0): 5.597981929779053, (6, 0, 4, 1): 5.4675374031066895, (2, 0, 2, 0): 5.957797527313232, (6, 1, 4, 0): 1.285017728805542, (5, 0, 5, 0): 5.563058376312256, (2, 1, 2, 1): 1.7716172933578491, (2, 0, 5, 0): 4.515780448913574, (3, 1, 3, 1): 3.690718173980713, (7, 0, 3, 0): 5.784040927886963, (3, 0, 5, 1): 1.1724662780761719, (5, 1, 3, 0): 4.493020057678223, (7, 0, 7, 0): 0.33898940682411194, (2, 0, 3, 1): 1.1808230876922607, (7, 1, 2, 0): 3.2570478916168213, (5, 1, 7, 0): 5.746870517730713, (7, 0, 2, 1): 4.771820545196533, (4, 0, 4, 0): 6.269706726074219, (3, 0, 7, 0): 5.21356725692749, (4, 1, 4, 1): 0.3871554136276245, (6, 0, 6, 0): 6.055601596832275

energy = -0.08528680 , angles= {(3, 1, 2, 0): 1.376722812652588, (5, 0, 7, 1): 2.962172269821167, (3, 0, 2, 1): 2.562309980392456, (5, 1, 2, 1): 2.237804412841797, (3, 1, 7, 1): 0.04895593598484993, (5, 0, 2, 0): 1.409030556678772, (2, 1, 7, 0): 3.2421014308929443, (4, 0, 6, 1): 5.5673723220825195, (7, 0, 5, 1): 4.326408386230469, (4, 1, 6, 0): 2.2725043296813965, (6, 0, 4, 1): 1.622938871383667, (2, 0, 2, 0): 1.542204737663269, (6, 1, 4, 0): 2.8526735305786133, (5, 0, 5, 0): 1.8852750062942505, (2, 1, 2, 1): 6.230629920959473, (2, 0, 5, 0): 0.6687994003295898, (3, 1, 3, 1): 6.109746932983398, (7, 0, 3, 0): 2.9365391731262207, (3, 0, 5, 1): 3.9655866622924805, (5, 1, 3, 0): 1.302794337272644, (7, 0, 7, 0): 0.23445840179920197, (2, 0, 3, 1): 4.712743759155273, (7, 1, 2, 0): 4.585455417633057, (5, 1, 7, 0): 1.0942513942718506, (7, 0, 2, 1): 6.081312656402588, (4, 0, 4, 0): 5.333253860473633, (3, 0, 7, 0): 2.693779706954956, (4, 1, 4, 1): 0.46065258979797363, (6, 0, 6, 0): 5.3260998725891

KeyboardInterrupt: 

We obtain a UCCSD energy within chemical accuracy (< 1.6 miliHartree from FCI energy) for the H$_2$O molecule near equilibrium geometry. As the bond distance approaches the dissociation limit, the energy deviation from FCI is typically expected to increase as electronic correlations increase during covalent bond-breaking. Furthermore, as HF becomes energetically more distant from the FCI wavefunction, the initial guess of all amplitudes being zero may lead to a local minimum. One can then instead initialize the amplitudes using random guesses, and repeat for $n$ samples to attempt to find the global minimum. To facilitate this, one can run the commented lines below. This will execute $n$ instances of tq.minimize for uniformly random initial parameters in interval $[0, 2\pi]$ and return the lowest optimization result.

In [ ]:
n=10
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

## Qubit Coupled Cluster (QCC)

In contrast to UCC, the QCC methodology makes no direct reference to fermionic algebra and seeks to construct an efficient ansatz directly in qubit-space by finding multi-qubit Pauli strings (entanglers) which lower energy. This is done through an energy-lowering heuristic employing the energy gradient with respect to a Pauli strings variational amplitude. As opposed to UCCSD, the circuit depth and number of parameter is chosen to meet hardware limitations, i.e. one must choose how many exponentiated Pauli strings will be entering the QCC ansatz.

### LiH in STO-3G basis

Below we perform the entangler screening protocol for LiH in minimal basis, and obtain one grouping of entanglers with non-zero energy gradient. We then select one of them to be used in the QCC VQE simulation.

In [18]:
xyz_data = get_molecular_data('lih', geometry=1.5, xyz_format=True)
basis='sto-3g'

lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set='sto-3g')

hf_reference = hf_occ(2*lih.n_orbitals, lih.n_electrons)

H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 1

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*lih.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*lih.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)




Hamiltonian has 15 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.2822

Selected entanglers:
1.0 [X0 Y1 X2 X3]


Once the QCC ranking procedure has been ran, we can simulate the QCC VQE optimization with the generated entanglers. The VQE optimization for the QCC ansatz is of the form
$$E = \min_{\boldsymbol{\Omega}, \boldsymbol{\tau}} \langle \boldsymbol{\Omega} | U_{\text{ENT}}^\dagger (\boldsymbol{\tau}) \hat H  U_{\text{ENT}} (\boldsymbol{\tau}) | \boldsymbol{\Omega} \rangle $$
where $\boldsymbol{\Omega}$ denote collective Euler angles parameterizing single-qubit rotations, and $\boldsymbol{\tau}$ are entangler amplitudes. 

In [19]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*lih.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-6, backend='qulacs_gpu')

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs_gpu
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 18 expectationvalues

active variables : 9

E=-0.70294360  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 0.0}  samples= None
E=-0.77938639  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 0.2822100520133972}  samples= None
E=-0.93592356  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 1.411050260066986}  samples= None
E=-0.93603455  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.

We see that the QCC energy converged to the FCI energy with only a single entangler! 

### H4 in 6-31G basis

Let us move on to the problem of H$_4$O in 6-31G basis using the same active space as the UCCSD example.

In [20]:
xyz_data = get_molecular_data('nh3', geometry=1, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
nh3 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals = active)
hf_reference = hf_occ(2*nh3.n_orbitals, nh3.n_electrons)


H = nh3.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 6

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*nh3.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*nh3.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.

Hamiltonian has 185 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.0758
2 : 0.0371
3 : 0.0371
4 : 0.031
5 : 0.0301
6 : 0.0249
7 : 0.0198
8 : 0.0198
9 : 0.0173
10 : 0.0173
11 : 0.0
12 : 0.0

Selected entanglers:
1.0 [X0 Y1 X6 X7]
1.0 [X0 Y3 X5 X6]
1.0 [X1 Y2 X4 X7]
1.0 [X2 Y3 X4 X5]
1.0 [X0 Y1 X4 X5]
1.0 [X2 Y3 X6 X7]


With the 6 chosen entanglers, we can then run the QCC VQE simulation.

In [21]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*nh3.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4, backend='qulacs_gpu')


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs_gpu
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 44 expectationvalues

active variables : 22

E=-75.98071144  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0, tau_1: 0.0, tau_2: 0.0, tau_3: 0.0, tau_4: 0.0, tau_5: 0.0}  samples= None
E=-75.98783364  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.075843811035156

We obtain chemical accuracy for water near equilibrium geometry with only 6 entanglers. The obtained energy is not as accurate as that of UCCSD for this problem, however the QCC optimization may be performed at a fraction of the UCCSD circuit depth. One can also increase the number of entanglers entering the QCC ansatz to increase accuracy. As a final check, one can always run $n$ VQE trials with random initial guesses to test if the optimization fell into a local minimum. ***(Warning: Completing n=10 trials may take a few minutes for this VQE instance).***

In [8]:
n = 10
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs_gpu
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 512 expectationvalues

active variables : 8

E=-74.57743409  angles= {(3, 0, 3, 0): 4.447969245977526, (3, 1, 2, 0): 0.3233918035331016, (2, 1, 2, 1): 5.383733402969034, (2, 0, 3, 1): 4.387805140966348, (3, 1, 3, 1): 2.5047312990537436, (3, 0, 2, 1): 0.46874659710961036, (2, 1, 3, 0): 5.33056600688194, (2, 0, 2, 0): 2.513692248490253}  samples= None
E=-74.66249491  angles= {(3, 0, 3, 0): 4.315194892950182, (3, 1, 2, 0): 0.4009407842459922, (2, 1, 2, 1): 5.621823918105752, (2, 0, 3, 1): 4.262578166479043, (3, 1, 3, 1): 2.2601252809873373, (3, 0, 2, 1): 0.66492694073754, (2, 1, 3, 0): 5.430713254196393, (2, 0, 2, 0): 2.3270543700722843}  samples= None
E=-74.67192535  angles= {(3, 0, 3, 0): 4.239942243573449, (3, 1, 2, 0): 0.38361525825944476,

E=-75.94560450  angles= {(3, 0, 3, 0): 3.135476543406907, (3, 1, 2, 0): -0.05482059846592682, (2, 1, 2, 1): 3.201220016045298, (2, 0, 3, 1): 4.441642509303768, (3, 1, 3, 1): 1.6611147109089885, (3, 0, 2, 1): -0.02515046448237812, (2, 1, 3, 0): 6.137072329510397, (2, 0, 2, 0): 1.5545457281766577}  samples= None
E=-75.96656084  angles= {(3, 0, 3, 0): 3.1303132507688463, (3, 1, 2, 0): -0.053031293330138854, (2, 1, 2, 1): 3.1637834066375095, (2, 0, 3, 1): 4.3636584831010214, (3, 1, 3, 1): 1.6213253838578545, (3, 0, 2, 1): -0.17310863073975682, (2, 1, 3, 0): 6.160483526472767, (2, 0, 2, 0): 1.5702773905210459}  samples= None
E=-75.97104870  angles= {(3, 0, 3, 0): 3.0926868227283504, (3, 1, 2, 0): -0.06564041374603383, (2, 1, 2, 1): 3.130566824363696, (2, 0, 3, 1): 4.534578822640642, (3, 1, 3, 1): 1.5797191402310342, (3, 0, 2, 1): -0.08386745412918915, (2, 1, 3, 0): 6.073021134957017, (2, 0, 2, 0): 1.597758776381497}  samples= None
E=-75.97851757  angles= {(3, 0, 3, 0): 3.0923765070667906, (

E=-75.92761455  angles= {(3, 0, 3, 0): 3.23739535504778, (3, 1, 2, 0): 4.369687751411669, (2, 1, 2, 1): 3.058107334352944, (2, 0, 3, 1): 1.9842641320543608, (3, 1, 3, 1): -0.04349245287458067, (3, 0, 2, 1): 2.4077848270233737, (2, 1, 3, 0): 3.9927686510746576, (2, 0, 2, 0): 6.213602508242691}  samples= None
E=-75.94835823  angles= {(3, 0, 3, 0): 3.1830214255019573, (3, 1, 2, 0): 4.365210711301197, (2, 1, 2, 1): 3.1554367168124497, (2, 0, 3, 1): 1.9929493108965595, (3, 1, 3, 1): 0.02236173537281691, (3, 0, 2, 1): 2.402199299307903, (2, 1, 3, 0): 3.99383432501939, (2, 0, 2, 0): 6.208687763051973}  samples= None
E=-75.97493693  angles= {(3, 0, 3, 0): 3.1195099791604686, (3, 1, 2, 0): 4.35289075063462, (2, 1, 2, 1): 3.098435470235046, (2, 0, 3, 1): 2.0319584765070706, (3, 1, 3, 1): 0.02687310721193481, (3, 0, 2, 1): 2.388640795664233, (2, 1, 3, 0): 4.052584245725779, (2, 0, 2, 0): 6.237407976073528}  samples= None
E=-75.96468797  angles= {(3, 0, 3, 0): 3.0258873050114325, (3, 1, 2, 0): 4.2

E=-75.98556753  angles= {(3, 0, 3, 0): 3.0588295442123337, (3, 1, 2, 0): 4.87261090863161, (2, 1, 2, 1): 6.264527818648984, (2, 0, 3, 1): 0.042900854730490476, (3, 1, 3, 1): 4.7222026587633446, (3, 0, 2, 1): 3.203956831193532, (2, 1, 3, 0): 3.2652453353489514, (2, 0, 2, 0): 4.745299890813806}  samples= None
E=-75.98700438  angles= {(3, 0, 3, 0): 3.0671083407888635, (3, 1, 2, 0): 4.87032699796129, (2, 1, 2, 1): 6.264507651186323, (2, 0, 3, 1): 0.04058171347664614, (3, 1, 3, 1): 4.726368077728879, (3, 0, 2, 1): 3.2067256162243285, (2, 1, 3, 0): 3.2667246097337594, (2, 0, 2, 0): 4.741201213312945}  samples= None
E=-75.98897744  angles= {(3, 0, 3, 0): 3.0806396305350017, (3, 1, 2, 0): 4.8624760542369, (2, 1, 2, 1): 6.2589241251668835, (2, 0, 3, 1): 0.033875495194583216, (3, 1, 3, 1): 4.728209171853047, (3, 0, 2, 1): 3.2163712973029406, (2, 1, 3, 0): 3.2739668425136785, (2, 0, 2, 0): 4.7348180398017226}  samples= None
E=-75.98932117  angles= {(3, 0, 3, 0): 3.09611973407434, (3, 1, 2, 0): 4.

E=-75.98935407  angles= {(3, 0, 3, 0): 6.234651655559132, (3, 1, 2, 0): 5.6558951431827, (2, 1, 2, 1): 3.1174128740202045, (2, 0, 3, 1): 2.376207481634754, (3, 1, 3, 1): 1.592294297671678, (3, 0, 2, 1): 3.020307069422432, (2, 1, 3, 0): 3.2230223851612636, (2, 0, 2, 0): 1.5886900857745339}  samples= None
E=-75.98935412  angles= {(3, 0, 3, 0): 6.234596744066044, (3, 1, 2, 0): 5.656007849576076, (2, 1, 2, 1): 3.1175201695864647, (2, 0, 3, 1): 2.376209412098322, (3, 1, 3, 1): 1.5922405450519157, (3, 0, 2, 1): 3.0203613883799463, (2, 1, 3, 0): 3.223220670809801, (2, 0, 2, 0): 1.5887751273327784}  samples= None
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs_gpu
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 512 expectationvalues

active variables : 8

E=-74.43833007  angles= {(3, 0, 3, 0): 3.9205888997314857, (3, 1, 2, 0): 2.4772460083178522,

...

E=-75.98935411  angles= {(3, 0, 3, 0): 0.04854412253752157, (3, 1, 2, 0): 3.225395642584036, (2, 1, 2, 1): 6.307298644161567, (2, 0, 3, 1): 1.6539987458843461, (3, 1, 3, 1): 3.1236519584875824, (3, 0, 2, 1): 3.176064823490569, (2, 1, 3, 0): 4.455218129027834, (2, 0, 2, 0): -0.021391564893789253}  samples= None
E=-75.98935413  angles= {(3, 0, 3, 0): 0.048601661108510844, (3, 1, 2, 0): 3.2253049676248686, (2, 1, 2, 1): 6.307276960088044, (2, 0, 3, 1): 1.6539980313830673, (3, 1, 3, 1): 3.1235984913799046, (3, 0, 2, 1): 3.1760506572736666, (2, 1, 3, 0): 4.455237180303483, (2, 0, 2, 0): -0.0214241396350879}  samples= None
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs_gpu
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 512 expectationvalues

active variables : 8

E=-74.81827680  angles= {(3, 0, 3, 0): 0.4075210971926113, (3, 1, 2, 0): 2.44013

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs_gpu
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 512 expectationvalues

active variables : 8

E=-74.83136441  angles= {(3, 0, 3, 0): 2.548610874111681, (3, 1, 2, 0): 3.205901653846054, (2, 1, 2, 1): 2.8008561265583194, (2, 0, 3, 1): 5.293788484835786, (3, 1, 3, 1): 4.373945788047658, (3, 0, 2, 1): 0.45514963679282694, (2, 1, 3, 0): 2.5477674375219856, (2, 0, 2, 0): 5.664008766636954}  samples= None
E=-75.68752665  angles= {(3, 0, 3, 0): 2.882602878506212, (3, 1, 2, 0): 3.2049899411995697, (2, 1, 2, 1): 2.8709244859333194, (2, 0, 3, 1): 5.273622087741059, (3, 1, 3, 1): 4.861754015586721, (3, 0, 2, 1): 0.4285936217781785, (2, 1, 3, 0): 2.5442006955786263, (2, 0, 2, 0): 4.88340726517211}  samples= None
E=-74.53850500  angles= {(3, 0, 3, 0): 4.045963048550297, (3, 1, 2, 0): 3.2348245161196845

E=-74.94465351  angles= {(3, 0, 3, 0): 3.643525376268138, (3, 1, 2, 0): 1.2963053084172798, (2, 1, 2, 1): 1.4026403982173072, (2, 0, 3, 1): 0.014090049744029505, (3, 1, 3, 1): 2.616852278842724, (3, 0, 2, 1): 0.8030999605189431, (2, 1, 3, 0): 1.3780174312310702, (2, 0, 2, 0): 6.251312960088511}  samples= None
E=-74.95741884  angles= {(3, 0, 3, 0): 3.6811683242269515, (3, 1, 2, 0): 1.2964593864967053, (2, 1, 2, 1): 1.3639074806357478, (2, 0, 3, 1): -0.01170352344451522, (3, 1, 3, 1): 2.691516685659151, (3, 0, 2, 1): 0.8118001279592509, (2, 1, 3, 0): 1.3720694641821463, (2, 0, 2, 0): 6.309070442080653}  samples= None
E=-74.98645815  angles= {(3, 0, 3, 0): 3.831740116062207, (3, 1, 2, 0): 1.2970756988144072, (2, 1, 2, 1): 1.20897581030951, (2, 0, 3, 1): -0.11487781619869414, (3, 1, 3, 1): 2.9901743129248572, (3, 0, 2, 1): 0.8466007977204827, (2, 1, 3, 0): 1.34827759598645, (2, 0, 2, 0): 6.540100370049221}  samples= None
E=-75.05468584  angles= {(3, 0, 3, 0): 3.9836662967756915, (3, 1, 2, 

E=-75.04645678  angles= {(3, 0, 3, 0): 0.00952117169184355, (3, 1, 2, 0): 1.8675920518816143, (2, 1, 2, 1): 4.015455475705078, (2, 0, 3, 1): 1.78814716728149, (3, 1, 3, 1): 3.2924191080386183, (3, 0, 2, 1): 1.2094214823475065, (2, 1, 3, 0): 5.24902619137897, (2, 0, 2, 0): 0.8074058613024832}  samples= None
E=-75.11454064  angles= {(3, 0, 3, 0): 0.3694111384509021, (3, 1, 2, 0): 1.813541953407161, (2, 1, 2, 1): 3.936185037376929, (2, 0, 3, 1): 1.7295353066855785, (3, 1, 3, 1): 3.479203094206264, (3, 0, 2, 1): 1.2566516567478636, (2, 1, 3, 0): 5.228414637044863, (2, 0, 2, 0): 0.3821453550230147}  samples= None
E=-75.22231484  angles= {(3, 0, 3, 0): 0.2774617699697489, (3, 1, 2, 0): 1.8311798420073837, (2, 1, 2, 1): 3.850121126425712, (2, 0, 3, 1): 1.7656844059012828, (3, 1, 3, 1): 3.4900993574523853, (3, 0, 2, 1): 1.1912218480563967, (2, 1, 3, 0): 5.193128209948424, (2, 0, 2, 0): 0.3307656660125926}  samples= None
E=-75.42823435  angles= {(3, 0, 3, 0): 0.17377983666266442, (3, 1, 2, 0): 

NameError: name 'entanglers' is not defined